<a href="https://colab.research.google.com/github/Ammad12345390/RAG_Chatbot-/blob/main/rag_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain-community langchain-openai faiss-cpu pypdf python-docx docx2txt openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.8/810.8 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.100.0
    Uninstalling openai-1.100.0:
      Successfully uninstalled openai-1.100.0


In [15]:
import os
from getpass import getpass
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from openai import OpenAI

In [16]:
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")


client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


embedding = OpenAIEmbeddings()


FOLDER_PATH = "./all_filesz"
INDEX_PATH = "./faiss_index"

Enter your OpenAI API key: ··········


In [17]:
FOLDER_PATH = "/content/all_filesz"
INDEX_PATH = "/content/faiss_index"

departments = {
    "compliance_and_legal": [
        "Contract Review Checklist",
        "GDPR Compliance Guidelines",
        "HIPAA Data Handling Policy"
    ],
    "marketing": [
        "Brand Guidelines Handbook",
        "Product Launch Campaign Report",
        "Q3 Content Marketing Strategy"
    ],
    "policies": [
        "Employee Leave & Absence Policy",
        "Remote Work & Hybrid Policy",
        "Workplace Code of Conduct"
    ],
    "technical_sops": [
        "Backend Service Monitoring SOP",
        "CI CD Deployment Pipeline Guide",
        "Production Incident Response Runbook"
    ]
}

embedding = OpenAIEmbeddings()

# ✅ Load existing FAISS index
if os.path.exists(f"{INDEX_PATH}/index.faiss"):
    print("✅ Loading existing FAISS index...")
    vectorstore = FAISS.load_local(INDEX_PATH, embedding, allow_dangerous_deserialization=True)

else:
    if not os.path.exists(FOLDER_PATH):
        os.makedirs(FOLDER_PATH)
        print(f"📂 Created '{FOLDER_PATH}'. Please add PDF/DOCX files and rerun.")
        raise SystemExit

    all_documents = []

    for file in os.listdir(FOLDER_PATH):
        path = os.path.join(FOLDER_PATH, file)
        file_name, ext = os.path.splitext(file)
        file_name = file_name.strip()

        matched_dept = ""
        matched_title = file_name

        # Normalize file name for matching
        normalized_file = file_name.lower().replace("_", " ").replace("-", " ")

        # 🔍 Match department and title
        for dept, docs in departments.items():
            for title in docs:
                if title.lower() == normalized_file:
                    matched_dept = dept
                    matched_title = title
                    break

        loader = None
        if ext.lower() == ".pdf":
            loader = PyPDFLoader(path)
        elif ext.lower() == ".docx":
            loader = Docx2txtLoader(path)
        else:
            continue

        try:
            documents = loader.load()

            for i, doc in enumerate(documents, 1):
                doc.metadata.update({
                    "source_file": file,
                    "chunk_number": i,
                    "page_number": i,
                    "department": matched_dept,
                    "title": matched_title
      })

                # ✅ Print info for each document
                print(f"📂 Department: {matched_dept}")
                print(f"📄 Title: {matched_title}")
                print(f"➡️ Source: {file}, Chunk: {i}\n")

            all_documents.extend(documents)

        except Exception as e:
            print(f"⚠️ Could not load {file}, skipping... ({e})")
            continue


📂 Department: marketing
📄 Title: Q3 Content Marketing Strategy
➡️ Source: Q3_Content_Marketing_Strategy.docx, Chunk: 1

📂 Department: marketing
📄 Title: Brand Guidelines Handbook
➡️ Source: Brand_Guidelines_Handbook.docx, Chunk: 1

📂 Department: technical_sops
📄 Title: Backend Service Monitoring SOP
➡️ Source: Backend_Service_Monitoring_SOP.docx, Chunk: 1

📂 Department: compliance_and_legal
📄 Title: Contract Review Checklist
➡️ Source: Contract_Review_Checklist.docx, Chunk: 1

📂 Department: compliance_and_legal
📄 Title: GDPR Compliance Guidelines
➡️ Source: GDPR_Compliance_Guidelines.docx, Chunk: 1

📂 Department: marketing
📄 Title: Product Launch Campaign Report
➡️ Source: Product_Launch_Campaign_Report.docx, Chunk: 1

📂 Department: 
📄 Title: Contract_Review_Checklist (1)
➡️ Source: Contract_Review_Checklist (1).docx, Chunk: 1

📂 Department: compliance_and_legal
📄 Title: HIPAA Data Handling Policy
➡️ Source: HIPAA_Data_Handling_Policy.docx, Chunk: 1

📂 Department: technical_sops
📄 Titl

In [18]:
if not all_documents:
    print(f"⚠️ No valid PDF/DOCX files found in '{FOLDER_PATH}'.")
    raise SystemExit

os.makedirs(INDEX_PATH, exist_ok=True)
vectorstore = FAISS.from_documents(all_documents, embedding)
vectorstore.save_local(INDEX_PATH)
print("✅ FAISS index created and saved with department + title metadata!")


✅ FAISS index created and saved with department + title metadata!


In [19]:
vectorstore = FAISS.load_local(INDEX_PATH, embedding, allow_dangerous_deserialization=True)

# Get the total number of documents
total_docs = len(vectorstore.docstore._dict)  # FAISS stores docs in docstore
print(f"Total documents in vectorstore: {total_docs}")

Total documents in vectorstore: 10


In [20]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(all_documents)


vectorstore = FAISS.from_documents(chunks, embedding)
vectorstore.save_local(INDEX_PATH)
print("✅ FAISS index created and saved!")

retriever = vectorstore.as_retriever()


policy_prompt = """
You are a Company Policy Assistant. Only answer questions using the information
provided in the company's policy documents. If the answer is not found in the
policy, say: "I cannot find that information in the policy."
Be concise, professional, and accurate.
"""

✅ FAISS index created and saved!


In [21]:
results = retriever.get_relevant_documents(
    "GDPR rules",
    filter={
        "$or": [
            {
                "department": "marketing",
                "title": {"$in": [
                    "Brand Guidelines Handbook",
                    "Product Launch Campaign Report",
                    "Q3 Content Marketing Strategy"
                ]}
            },
            {
                "department": "policies",
                "title": {"$in": [
                    "Employee Leave & Absence Policy",
                    "Remote Work & Hybrid Policy",
                    "Workplace Code of Conduct"
                ]}
            },
            {
                "department": "technical sops",
                "title": {"$in": [
                    "Backend Service Monitoring SOP",
                    "CI/CD Deployment Pipeline Guide",
                    "Production Incident Response Runbook"
                ]}
            },
            {
                "department": "compliance_and_legal",
                "title": {"$in": [
                    "Contract Review Checklist",
                    "GDPR Compliance Guidelines",
                    "HIPAA Data Handling Policy"
                ]}
            }
        ]
    }
)

for doc in results:
    print(f"📂 Department: {doc.metadata.get('department')}")
    print(f"📄 Title: {doc.metadata.get('title')}")
    print(f"Content: {doc.page_content[:60]}...\n")


/tmp/ipython-input-1758526566.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(


📂 Department: compliance_and_legal
📄 Title: GDPR Compliance Guidelines
Content: GDPR Compliance Guidelines

Department: Legal

Jurisdiction:...

📂 Department: compliance_and_legal
📄 Title: GDPR Compliance Guidelines
Content: Page 2

User Consent Requirements
- Consent must be freely g...

📂 Department: compliance_and_legal
📄 Title: GDPR Compliance Guidelines
Content: Page 4

Data Breach & Penalties
- Notify supervisory authori...

📂 Department: compliance_and_legal
📄 Title: HIPAA Data Handling Policy
Content: HIPAA Data Handling Policy

Department: Legal

Jurisdiction:...



In [ ]:
print("Chatbot ready! Type 'exit' to stop.")

while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        print("Goodbye!")
        break

    k = 1
    relevant_docs = retriever.get_relevant_documents(query)[:k]

    # Combine content and metadata for context
    context_list = []
    for i, doc in enumerate(relevant_docs, 1):
        context_list.append(
            f"Chunk {i} Content:\n{doc.page_content}\nMetadata: {doc.metadata}\n"
        )
    context = "\n".join(context_list)


    print("\n=== Retrieved Chunks ===")
    for doc in relevant_docs:
        print("Content:", doc.page_content[:40], "...")
        print("Metadata:", doc.metadata)
        print("-" * 30)

    messages = [
        {"role": "system", "content": policy_prompt},
        {"role": "user", "content": f"Context from documents:\n{context}\n\nQuestion: {query}"}
    ]

    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=messages,
        max_tokens=300,
        temperature=0.7
    )

    print("Bot:", response.choices[0].message.content)

Chatbot ready! Type 'exit' to stop.
You: what is Overview of CI/CD

=== Retrieved Chunks ===
Content: CI/CD Deployment Pipeline Guide

Departm ...
Metadata: {'source': '/content/all_filesz/CI-CD_Deployment_Pipeline_Guide.docx', 'source_file': 'CI-CD_Deployment_Pipeline_Guide.docx', 'chunk_number': 1, 'page_number': 1, 'department': 'technical_sops', 'title': 'CI CD Deployment Pipeline Guide'}
------------------------------
Bot: The overview of CI/CD describes the automated workflow for deploying microservices using tools such as GitHub Actions, Docker, Kubernetes, Helm, and ArgoCD. It states that staging and production environments are deployed separately and that all pipelines are managed via YAML configuration files in the root directory.
